In [1]:

#!pip install transformers librosa soundfile numpy"<"2 torch torchaudio fast-tsp kmedoids


# Setup

In [14]:
%load_ext autoreload
%autoreload 2

from clap_slice import AudioOrderer, CLAPWrapper

import torch
torch.set_grad_enabled(False)

clap = CLAPWrapper()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
initilizing CLAP


# Run

In [15]:
filename = '/Users/damian/2.current/clapSlice/outputs/Dua Lipa - Be The One (Official Music Video).mp3'
bpm = 87.5

audio_orderer = AudioOrderer(clap=clap, source_audio_path=filename, bpm=bpm)

loaded waveform with shape torch.Size([2, 9805880]) , sampling rate 48000


In [4]:
chunk_beats = 2
window_width = 0
sort_order = audio_orderer.make_order(chunk_beats=chunk_beats, preserve_start_and_end=True)
sort_order

computing distance matrix
computing route


AudioOrdering(source_audio='/Users/damian/2.current/clapSlice/outputs/Dua Lipa - Be The One (Official Music Video).mp3', bpm=87.5, chunk_beats=2, sort_order=tensor([  0,   1,   2,   3,   4,   6,   5,  38,  95,  39,  94,  93,  37,  22,
         21,  13,  14,  15,  16,   8,   9,  17,  20,  73,  72,  74,  66,  76,
         68,  64,  65,  25,  81,  89,  88,  24,  80,  90,  34,  82,  26,  28,
         84,  33,  32,  79,  99, 131,  43, 143,  55, 101, 133,  45,  47, 135,
        103,  41,  97, 129,  40,  96,  98,  42, 130,  35,  91, 100,  44, 132,
        134,  46, 102,  75,  71,  67,  31,  87,  27,  83,  11, 123, 115, 126,
         19,  36,  92, 112, 117, 113, 114, 118, 121, 125, 120, 124, 116, 119,
        128,  51, 139, 107, 108, 140,  52,  48, 104, 136, 138, 105, 137,  49,
        141, 109,  53,  54, 142, 106,  50, 110,  78,  77,  69,  85,  29,  30,
         86,  70,  60,  56,  57,  62,  61,  58,  59,  63,  18, 122,  12,  10,
         23, 111, 127,   7, 144, 145, 146, 147]), window_width=

In [5]:
sort_order.source_audio = filename
sort_order.bpm = bpm
from clap_slice.audio_ordering_candidates_registry import save_candidate
#save_candidate(sort_order)

In [6]:
from clap_slice import SmearModifier

smear_modifiers = [
    SmearModifier(smear_width=1, spread=4, match_embedding=clap.get_text_features("vocal, song, emotional singing")),
    SmearModifier(smear_width=2, spread=1, match_embedding=clap.get_text_features("instrumental"))
]

In [7]:


audio_ordering_result = audio_orderer.apply_order(
    audio_ordering=sort_order,
    smear_width=2,
    spread=0,
    wrap_mode='bleed',
    save=True,
    smear_modifiers=smear_modifiers,
    smooth_smear_modifiers=True
)
!open outputs
#Audio(result.output_audio, rate=audio_orderer.sampling_rate)
#print(ordered.shape)

[0, 1, 2, 3, 4, 6, 5, 38, 95, 39, 94, 93, 37, 22, 21, 13, 14, 15, 16, 8, 9, 17, 20, 73, 72, 74, 66, 76, 68, 64, 65, 25, 81, 89, 88, 24, 80, 90, 34, 82, 26, 28, 84, 33, 32, 79, 99, 131, 43, 143, 55, 101, 133, 45, 47, 135, 103, 41, 97, 129, 40, 96, 98, 42, 130, 35, 91, 100, 44, 132, 134, 46, 102, 75, 71, 67, 31, 87, 27, 83, 11, 123, 115, 126, 19, 36, 92, 112, 117, 113, 114, 118, 121, 125, 120, 124, 116, 119, 128, 51, 139, 107, 108, 140, 52, 48, 104, 136, 138, 105, 137, 49, 141, 109, 53, 54, 142, 106, 50, 110, 78, 77, 69, 85, 29, 30, 86, 70, 60, 56, 57, 62, 61, 58, 59, 63, 18, 122, 12, 10, 23, 111, 127, 7, 144, 145, 146, 147]
smear width: 2.085270404815674  spread: 0.7441884279251099 -> 2 1
smear width: 2.086822271347046  spread: 0.7395330667495728 -> 2 1
smear width: 2.2614548206329346  spread: 0.21563565731048584 -> 2 1
smear width: 2.113037586212158  spread: 0.6608871221542358 -> 2 1
smear width: 2.078572988510132  spread: 0.7642813324928284 -> 2 1
smear width: 2.0363149642944336  spre

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Video

In [ ]:
from clap_slice import VideoChunkCache
from clap_slice import VideoWriter, apply_audio_smear_to_video
from pytorchvideo.data.encoded_video import EncodedVideo

video_in_filename = 'outputs/Dua Lipa - Be The One (Official Music Video) (480p_25fps_H264-128kbit_AAC).mp4'

video = EncodedVideo.from_path(video_in_filename, decode_audio=False)
video_chunk_cache = VideoChunkCache(video=video,
                                    chunk_size_seconds=audio_orderer.get_chunk_size_seconds(sort_order.chunk_beats),
                                    max_cache_size=30)

In [13]:
blend_mode = 'max'
video_writer = VideoWriter(output_path=f'outputs/Dua Lipa - Be The One (Official Music Video).mp3-sorted-bpm87.5-cb2-ww0-smeared-dyn.wav.mp3.{blend_mode}-sqrt2.mp4', fps=25, width=720, height=480)

apply_audio_smear_to_video(video_chunk_cache,
                           video_writer,
                           smear_details=audio_ordering_result.smear_details,
                           blend_mode=blend_mode,
                           max_chunks_to_write=None)

video_writer.close()

normalization_scaling: 0.4870553945448325


  0%|          | 0/152 [00:00<?, ?it/s]

loaded chunk 22
loaded chunk 11
loaded chunk 23
loaded chunk 12
loaded chunk 24
loaded chunk 13
evicting chunk 0
loaded chunk 14
evicting chunk 1
loaded chunk 15
evicting chunk 2
loaded chunk 16
evicting chunk 3
loaded chunk 17
evicting chunk 4
loaded chunk 8
evicting chunk 5
loaded chunk 9
evicting chunk 91
loaded chunk 18
evicting chunk 97
loaded chunk 10
evicting chunk 40
loaded chunk 71
evicting chunk 92
loaded chunk 72
evicting chunk 35
loaded chunk 70
evicting chunk 41
loaded chunk 73
evicting chunk 93
loaded chunk 74
evicting chunk 36
loaded chunk 64
evicting chunk 96
loaded chunk 75
evicting chunk 94
loaded chunk 65
evicting chunk 37
loaded chunk 66
evicting chunk 95
loaded chunk 76
evicting chunk 38
loaded chunk 67
evicting chunk 39
loaded chunk 62
evicting chunk 24
loaded chunk 77
evicting chunk 13
loaded chunk 68
evicting chunk 23
loaded chunk 63
evicting chunk 14
loaded chunk 78
evicting chunk 6
loaded chunk 69
evicting chunk 7
loaded chunk 23
evicting chunk 8
loaded chunk 